In [8]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('train.csv')

def preprocess_menu_data(df: pd.DataFrame):
    df = df.copy()

    # ---------------------------------------------------------
    # 1. 텍스트 정제 및 컬럼 분리
    key_col = '영업장명_메뉴명'

    # 특수 공백 제거 및 문자열 변환
    df[key_col] = df[key_col].astype(str).str.replace(r"\ufeff|\u200b", "", regex=True)
    df[key_col] = df[key_col].str.replace("\xa0", " ", regex=False).str.strip()

    # '_' 기준으로 업장명과 메뉴명 분리
    split_data = df[key_col].str.split('_', n=1, expand=True)
    df['업장명'] = split_data[0].str.strip()

    # 메뉴명이 없는 경우 처리
    if split_data.shape[1] > 1:
        df['메뉴명'] = split_data[1].str.strip()
    else:
        df['메뉴명'] = "미상"

    # ---------------------------------------------------------
    # 2. 메뉴 카테고리화
    def categorize_menu(name):
        name = str(name).lower().replace(" ", "")

        categories = {
            '행사/단체': ['단체', '연회', 'conference', 'ballroom', 'convention', 'opus', 'hall', 'platter', '패키지', '세트', 'openfood', '오픈푸드', '무제한', '뷔페', '정식'],
            '대여/공간': ['대여', '이용료', '렌탈', '의자', '룸', '잔디그늘집'],
            '주류': ['소주', '맥주', '와인', '막걸리', '하이볼', '칵테일', '카스', '테라', '참이슬', '처음처럼', '하이네켄', '버드와이저', '스텔라', '복분자', 'beer', 'wine', 'gls', 'bottle', '샷'],
            '음료/카페': ['아메리카노', '라떼', '에이드', '주스', '식혜', '차', '커피', '티', '콜라', '사이다', '스프라이트', '미숫가루', '생수', '토닉', 'coffee'],
            '구이/BBQ': ['bbq', '삼겹', '구이', '갈비', '목살', '양갈비', '한우', '등심', '스테이크', 'aus', '돈육', '소세지', '킬바사'],
            '면류': ['파스타', '스파게티', '짜장', '짬뽕', '우동', '국수', '냉면', '라면', '소바', '알리오', '까르보나라', '메밀'],
            '국/탕/찌개': ['찌개', '탕', '국밥', '전골', '해장국', '설렁탕', '갱시기', '육개장', '미역국'],
            '분식/튀김': ['떡볶이', '튀김', '돈까스', '핫도그', '치킨', '너겟', '어묵', '순대'],
            '밥류/한식': ['비빔밥', '덮밥', '볶음밥', '리조또', '공깃밥', '햇반', '주먹밥', '김치', '된장', '반찬'],
            '디저트': ['아이스크림', '케이크', '쿠키', '빵', '와플', '빙수', '디저트', '푸딩'],
            '소모품/기타': ['컵', '종이컵', '수저', '젓가락', '접시', '일회용', '가위', '집게', '부탄가스', '쌈장', '소스', '사리', '야채추가']
        }

        for cat, keywords in categories.items():
            for kw in keywords:
                if kw in name:
                    return cat
        return '기타'

    df['메뉴_카테고리'] = df['메뉴명'].apply(categorize_menu)
    return df

def encode_features_train_only(df: pd.DataFrame):
    df = df.copy()

    le_store = LabelEncoder()
    le_menu = LabelEncoder()
    le_cat = LabelEncoder()

    # Train 데이터에 대해서만 fit_transform 수행
    df['업장명_le'] = le_store.fit_transform(df['업장명'].astype(str))
    df['메뉴명_le'] = le_menu.fit_transform(df['메뉴명'].astype(str))
    df['카테고리_le'] = le_cat.fit_transform(df['메뉴_카테고리'].astype(str))

    return df, le_store, le_menu, le_cat


# 전처리 수행
train = preprocess_menu_data(df)

# 인코딩
train_encoded, le_store, le_menu, le_cat = encode_features_train_only(train)

In [9]:
# 결과
print(train_encoded[['영업장명_메뉴명', '업장명', '메뉴명', '메뉴_카테고리', '카테고리_le']].head())

             영업장명_메뉴명         업장명      메뉴명 메뉴_카테고리  카테고리_le
0  느티나무 셀프BBQ_1인 수저세트  느티나무 셀프BBQ  1인 수저세트   행사/단체       11
1  느티나무 셀프BBQ_1인 수저세트  느티나무 셀프BBQ  1인 수저세트   행사/단체       11
2  느티나무 셀프BBQ_1인 수저세트  느티나무 셀프BBQ  1인 수저세트   행사/단체       11
3  느티나무 셀프BBQ_1인 수저세트  느티나무 셀프BBQ  1인 수저세트   행사/단체       11
4  느티나무 셀프BBQ_1인 수저세트  느티나무 셀프BBQ  1인 수저세트   행사/단체       11
